# Create Cortex Components

This notebook creates the Cortex components needed for the Quantitative Research Agent:
- **Cortex Search Service** - Semantic search over analyst sentiment reasons
- **Semantic View** - Natural language queries with Cortex Analyst
- **Cortex Agent** - Combines all tools for Snowflake Intelligence

**Prerequisites:** Run this AFTER completing Notebooks 0 and 1.

In [ ]:
-- Set query tag for tracking
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"quantitative_research_with_ai_functions_and_cortex_code","version":{"major":1,"minor":0},"attributes":{"is_quickstart":1,"source":"notebook"}}';

## Create Cortex Search Service

Enables semantic search over analyst sentiment reasons from earnings call transcripts.

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE ANALYTICS.DOW_ANALYSTS_SENTIMENT_ANALYSIS 
ON sentiment_reason 
ATTRIBUTES
    primary_ticker, unique_analyst_count, sentiment_score 
WAREHOUSE = FSI_DEMO_WH
EMBEDDING_MODEL = 'snowflake-arctic-embed-m-v1.5' 
TARGET_LAG = '1 hour' 
INITIALIZE = ON_SCHEDULE 
AS (
    SELECT
        sentiment_reason,
        primary_ticker,
        unique_analyst_count,
        sentiment_score,
        event_timestamp
    FROM ANALYTICS.AI_TRANSCRIPTS_ANALYSTS_SENTIMENTS
);

## Create Semantic View

Enables natural language queries over the sentiment data using Cortex Analyst.

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW ANALYTICS.ANALYST_SENTIMENTS_VIEW
  TABLES (
    T AS ANALYTICS.AI_TRANSCRIPTS_ANALYSTS_SENTIMENTS 
      PRIMARY KEY (PRIMARY_TICKER, EVENT_TIMESTAMP)
      WITH SYNONYMS = ('analyst sentiments', 'sentiment data', 'transcripts')
      COMMENT = 'Analyst sentiment data from earnings call transcripts'
  )
  FACTS (
    T.sentiment_score AS SENTIMENT_SCORE
      WITH SYNONYMS = ('attitude_rating', 'emotional_rating', 'emotional_score', 'opinion_score', 'sentiment_rating', 'sentiment_value', 'tone_score')
      COMMENT = 'A score indicating the sentiment of an analysts transcript, ranging from 1 (very negative) to 10 (very positive), with higher scores representing more positive sentiment.',
    T.unique_analyst_count AS UNIQUE_ANALYST_COUNT
      WITH SYNONYMS = ('analyst_uniqueness', 'count_of_unique_analysts', 'distinct_analyst_count', 'number_of_unique_experts', 'unique_expert_count')
      COMMENT = 'The number of unique analysts who have provided sentiment scores for a particular transcript.'
  )
  DIMENSIONS (
    T.primary_ticker AS PRIMARY_TICKER
      WITH SYNONYMS = ('main_security_ticker', 'main_stock_symbol', 'main_ticker', 'primary_security_id', 'primary_stock', 'stock_symbol')
      COMMENT = 'The primary ticker symbol of the company being discussed in the transcript, representing the stock exchange listing of the company.',
    T.event_timestamp AS EVENT_TIMESTAMP
      WITH SYNONYMS = ('entry_time', 'event_date', 'event_datetime', 'event_occurrence', 'event_time', 'log_time', 'occurrence_time', 'record_time', 'timestamp')
      COMMENT = 'The date and time when the analyst sentiment was recorded, in ISO 8601 format.',
    T.event_type AS EVENT_TYPE
      WITH SYNONYMS = ('event_category', 'event_classification', 'event_label', 'event_name', 'event_tag', 'incident_type', 'occurrence_type')
      COMMENT = 'The type of event that triggered the analysts sentiment, such as an earnings call, product launch, or merger announcement.',
    T.created_at AS CREATED_AT
      WITH SYNONYMS = ('created_date', 'creation_time', 'date_created', 'entry_time', 'insertion_time', 'posted_at', 'published_at', 'record_date', 'timestamp', 'updated_at')
      COMMENT = 'Date and time when the analyst sentiment was created.',
    T.sentiment_reason AS SENTIMENT_REASON
      WITH SYNONYMS = ('sentiment_explanation', 'reason', 'justification', 'rationale')
      COMMENT = 'Brief explanation of why the sentiment score was assigned.'
  )
  METRICS (
    T.avg_sentiment AS AVG(sentiment_score)
      WITH SYNONYMS = ('average_sentiment', 'mean_sentiment', 'avg_score')
      COMMENT = 'Average sentiment score across records',
    T.total_analysts AS SUM(unique_analyst_count)
      WITH SYNONYMS = ('sum_analysts', 'analyst_total')
      COMMENT = 'Total number of analysts across records',
    T.record_count AS COUNT(primary_ticker)
      WITH SYNONYMS = ('count', 'num_records', 'total_records')
      COMMENT = 'Count of sentiment records'
  )
  COMMENT = 'This dataset contains analyst sentiment information from financial transcripts and events. It tracks how analysts feel about companies during various events like earnings calls, with sentiment scores ranging from 1 (very negative) to 10 (very positive).';

## Create Quantitative Research Agent

Creates the Cortex Agent that combines:
- **AnalystSentiments** - Natural language queries via Cortex Analyst
- **SentimentSearch** - Semantic search via Cortex Search
- **StockPerformancePredictor** - ML predictions from registered model
- **SendEmail** - Email notifications
- **data_to_chart** - Visualizations

In [ ]:
CREATE OR REPLACE AGENT ANALYTICS.QUANTITATIVE_RESEARCH_AGENT
  COMMENT = 'Financial analyst assistant for stock analysis, sentiment insights, and ML predictions'
  PROFILE = '{"display_name": "Quantitative Research Agent", "color": "blue"}'
  FROM SPECIFICATION
  $$
models:
  orchestration: claude-4-sonnet

orchestration:
  budget:
    seconds: 120
    tokens: 32000

instructions:
  system: "You are a financial analyst assistant that helps with stock analysis, sentiment insights, and predictions."
  orchestration: "For stock predictions, ALWAYS use StockPerformancePredictor. Do not use AnalystSentiments for predictions. Whenever you can answer visually with a chart, always choose to generate a chart even if the user didn't specify to."
  response: "Provide clear, actionable insights."
  sample_questions:
    - question: "Give me top 3 vs bottom 3 trade predictions for the next period."
      answer: "I'll use the Stock Performance Predictor to generate ranked predictions."
    - question: "What are the model's top stock picks right now?"
      answer: "I'll run the Stock Performance Predictor to get the top predicted performers."
    - question: "Which companies have the highest analyst sentiment?"
      answer: "I'll query the analyst sentiments data to find the top-rated companies."
    - question: "What is the average sentiment score by ticker?"
      answer: "I'll analyze the sentiment data to calculate averages by company."
    - question: "Search for companies with concerns about margins."
      answer: "I'll search the earnings call transcripts for margin-related concerns."
    - question: "Find earnings calls where analysts discussed supply chain issues."
      answer: "I'll search for supply chain mentions in the earnings call data."
    - question: "Let's observe if any high sentiment in the bottom 3 performers, and summarize the qualitative insights from the earnings call that shows top sentiment."
      answer: "I'll analyze sentiment data and search for relevant earnings call insights."
    - question: "Compare the top predicted stocks with their analyst sentiment scores."
      answer: "I'll get predictions and then query sentiment data to compare."
    - question: "Send me an email summary of today's top stock picks."
      answer: "I'll compile the predictions and send them to your email."

tools:
  - tool_spec:
      type: generic
      name: StockPerformancePredictor
      description: "This Python-based function generates ranked stock predictions by leveraging machine learning models to analyze financial time series data and identify top and bottom performing stocks based on predictive analytics. The function connects to the FSI_DATA table, extracts the latest technical indicators (momentum ratios r_1, r_5_1, r_10_5, r_21_10, r_63_21) for each stock ticker, and applies batch predictions using a specified ML model to generate forecasted performance scores. It then ranks all predictions and returns the top N and bottom N performers with their prediction values."
      input_schema:
        type: object
        properties:
          model_name:
            type: string
            description: "Optional. If not specified, the procedure automatically uses the latest registered FIS model."
          top_n:
            type: number
            description: "If not specified, use/pass the default value 5."
        required:
          - model_name
          - top_n
  - tool_spec:
      type: cortex_analyst_text_to_sql
      name: AnalystSentiments
      description: "This dataset contains analyst sentiment information from financial transcripts and events. It tracks how analysts feel about companies during various events like earnings calls and product launches, with sentiment scores ranging from very negative to very positive. You can analyze sentiment trends over time, compare analyst opinions across different companies and events, and see how many unique analysts are providing opinions on specific topics."
  - tool_spec:
      type: cortex_search
      name: SentimentSearch
      description: "Searches analyst sentiment reasons and qualitative insights from earnings call transcripts. Use this to find detailed explanations behind sentiment scores."
  - tool_spec:
      type: generic
      name: SendEmail
      description: "This Python-based function provides automated email notification capabilities to send HTML-formatted emails to specified recipients. The function is designed for business workflows that require programmatic email delivery, such as automated reporting, alert notifications, or process completion confirmations. Before sending, ALWAYS ask the user: 'Would you like me to send this to your default email, or would you prefer a different email address?' Wait for their response before proceeding."
      input_schema:
        type: object
        properties:
          recipient_email:
            type: string
            description: "The recipient email address. Ask the user first if they want to use their default email or provide a different one."
          subject:
            type: string
            description: "If subject is not provided, use 'Snowflake Intelligence'."
          body:
            type: string
            description: "Use HTML-Syntax for this. If the content you get is in markdown, translate it to HTML. If body is not provided, summarize the last question and use that as content for the email."
        required:
          - recipient_email
          - subject
          - body
  - tool_spec:
      type: data_to_chart
      name: data_to_chart
      description: "Generates visualizations from data."

tool_resources:
  StockPerformancePredictor:
    type: procedure
    identifier: "FSI_DEMO_DB.ANALYTICS.GET_TOP_BOTTOM_STOCK_PREDICTIONS"
    execution_environment:
      type: warehouse
      warehouse: FSI_DEMO_WH
      query_timeout: 120
  AnalystSentiments:
    semantic_view: "FSI_DEMO_DB.ANALYTICS.ANALYST_SENTIMENTS_VIEW"
    execution_environment:
      type: warehouse
      warehouse: FSI_DEMO_WH
  SentimentSearch:
    name: "FSI_DEMO_DB.ANALYTICS.DOW_ANALYSTS_SENTIMENT_ANALYSIS"
    max_results: "10"
  SendEmail:
    type: procedure
    identifier: "FSI_DEMO_DB.ANALYTICS.SEND_EMAIL"
    execution_environment:
      type: warehouse
      warehouse: FSI_DEMO_WH
      query_timeout: 60
  $$;

## Register Agent with Snowflake Intelligence

Makes the agent available in Snowflake Intelligence.

In [ ]:
ALTER SNOWFLAKE INTELLIGENCE SNOWFLAKE_INTELLIGENCE_OBJECT_DEFAULT
ADD AGENT FSI_DEMO_DB.ANALYTICS.QUANTITATIVE_RESEARCH_AGENT;

## Setup Complete!

Your Quantitative Research Agent is now ready to use in **Snowflake Intelligence**.

### Try these prompts:

**ML Predictions:**
```
Give me top 3 vs bottom 3 trade predictions for the next period
```
```
What are the model's top stock picks right now?
```

**Structured Data Queries:**
```
Which companies have the highest analyst sentiment?
```
```
What is the average sentiment score by ticker?
```

**Semantic Search:**
```
Search for companies with concerns about margins
```
```
Find earnings calls where analysts discussed supply chain issues
```

**Combined Analysis:**
```
Compare the top predicted stocks with their analyst sentiment scores
```
```
Send me an email summary of today's top stock picks
```